In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.6 MB/s eta 0:00:00


In [3]:
!pip install torch-geometric

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [5]:
# Step 3: Make a tiny social network
import torch
from torch_geometric.data import Data

# 6 Indian friends (nodes) with random features
names = ["Amit", "Priya", "Ravi", "Neha", "Arjun", "Sneha"]

x = torch.rand(6, 3)   # 6 nodes, each has 3 features

# Friendships (edges)
# Example: Amit is friends with Priya, Amit is friends with Ravi, etc.
edge_index = torch.tensor([[0, 0, 1, 2, 3, 4],
                           [1, 2, 2, 3, 4, 5]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)
print(data)

# Step 4: Build a simple GNN model
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# GNN Encoder
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)  # Node embeddings

# Step 5: Link Prediction (Friend Recommendation)
class LinkPredictor(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.encoder = GCN(in_channels, 16, 8)

    def forward(self, x, edge_index, edge_pairs):
        z = self.encoder(x, edge_index)
        z_u, z_v = z[edge_pairs[0]], z[edge_pairs[1]]
        return torch.sigmoid((z_u * z_v).sum(dim=-1))

# Step 6: Train the model
import itertools

model = LinkPredictor(3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Positive edges (real friendships)
pos_edges = edge_index

# Negative edges (random non-friends)
all_pairs = torch.tensor(list(itertools.combinations(range(6), 2)))
mask = torch.ones(len(all_pairs), dtype=torch.bool)
for e in pos_edges.t():
    mask[(all_pairs[:,0]==e[0]) & (all_pairs[:,1]==e[1])] = False
neg_edges = all_pairs[mask][:len(pos_edges.t())]  # same count as positive

for epoch in range(50):
    model.train()
    optimizer.zero_grad()

    # Positive predictions
    pos_pred = model(data.x, data.edge_index, pos_edges)
    pos_loss = F.binary_cross_entropy(pos_pred, torch.ones(pos_pred.size(0)))

    # Negative predictions
    neg_pred = model(data.x, data.edge_index, neg_edges.t())
    neg_loss = F.binary_cross_entropy(neg_pred, torch.zeros(neg_pred.size(0)))

    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Step 7: Recommend new friends 🎉
model.eval()
scores = model(data.x, data.edge_index, neg_edges.t())

for (u,v), score in zip(neg_edges, scores):
    print(f"Suggest friendship between {names[u.item()]} and {names[v.item()]} with score {score.item():.2f}")


Data(x=[6, 3], edge_index=[2, 6])
Epoch 1, Loss: 1.3497
Epoch 2, Loss: 1.3270
Epoch 3, Loss: 1.3224
Epoch 4, Loss: 1.3129
Epoch 5, Loss: 1.2995
Epoch 6, Loss: 1.2887
Epoch 7, Loss: 1.2815
Epoch 8, Loss: 1.2720
Epoch 9, Loss: 1.2577
Epoch 10, Loss: 1.2421
Epoch 11, Loss: 1.2284
Epoch 12, Loss: 1.2138
Epoch 13, Loss: 1.1970
Epoch 14, Loss: 1.1783
Epoch 15, Loss: 1.1578
Epoch 16, Loss: 1.1368
Epoch 17, Loss: 1.1153
Epoch 18, Loss: 1.0909
Epoch 19, Loss: 1.0629
Epoch 20, Loss: 1.0326
Epoch 21, Loss: 1.0000
Epoch 22, Loss: 0.9661
Epoch 23, Loss: 0.9294
Epoch 24, Loss: 0.8908
Epoch 25, Loss: 0.8541
Epoch 26, Loss: 0.8154
Epoch 27, Loss: 0.7737
Epoch 28, Loss: 0.7297
Epoch 29, Loss: 0.6856
Epoch 30, Loss: 0.6416
Epoch 31, Loss: 0.5990
Epoch 32, Loss: 0.5595
Epoch 33, Loss: 0.5241
Epoch 34, Loss: 0.4915
Epoch 35, Loss: 0.4623
Epoch 36, Loss: 0.4364
Epoch 37, Loss: 0.4140
Epoch 38, Loss: 0.3945
Epoch 39, Loss: 0.3777
Epoch 40, Loss: 0.3636
Epoch 41, Loss: 0.3523
Epoch 42, Loss: 0.3437
Epoch 43,